# Model
In this notebook, we:
- Define the structure of our prediction model.
- Try different models and assess their performance.
- Predict on the 2024 March Madness bracket.

## Imports

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score, log_loss, accuracy_score, confusion_matrix, classification_report
import xgboost as xgb

# display 100 rows and 100 columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# global random seed
SEED = 9

## Load Data

In [5]:
# load in features df
fcomp = pd.read_csv('data/processed/features_compact.csv')
# fdet = pd.read_csv('data/processed/features_detailed.csv')

## ML Model

In [9]:
fcomp

,A_1_pos_game_ratio,A_1_pos_losses,A_1_pos_win_ratio,A_1_pos_wins,A_Loc,A_PlayIn,A_Region,A_Seed,A_TeamID,A_avg_loss_diff,A_avg_pts_against_loss,A_avg_pts_against_win,A_avg_pts_for_loss,A_avg_pts_for_win,A_avg_win_diff,A_away_losses,A_away_win_ratio,A_away_wins,A_home_losses,A_home_win_ratio,A_home_wins,A_max_loss_diff,A_max_win_diff,A_neutral_losses,A_neutral_win_ratio,A_neutral_wins,A_num_losses,A_num_wins,A_ot_losses,A_ot_ratio,A_ot_win_ratio,A_ot_wins,A_recent_avg_pts_against,A_recent_avg_pts_for,A_recent_avg_score_diff,A_recent_losses,A_recent_pts_against_loss,A_recent_pts_against_win,A_recent_pts_for_loss,A_recent_pts_for_win,A_recent_win_ratio,A_recent_wins,A_std_loss_diff,A_std_pts_against_loss,A_std_pts_against_win,A_std_pts_for_loss,A_std_pts_for_win,A_std_win_diff,A_win_ratio,B_1_pos_game_ratio,...,B_1_pos_win_ratio,B_1_pos_wins,B_PlayIn,B_Region,B_Seed,B_TeamID,B_avg_loss_diff,B_avg_pts_against_loss,B_avg_pts_against_win,B_avg_pts_for_loss,B_avg_pts_for_win,B_avg_win_diff,B_away_losses,B_away_win_ratio,B_away_wins,B_home_losses,B_home_win_ratio,B_home_wins,B_max_loss_diff,B_max_win_diff,B_neutral_losses,B_neutral_win_ratio,B_neutral_wins,B_num_losses,B_num_wins,B_ot_losses,B_ot_ratio,B_ot_win_ratio,B_ot_wins,B_recent_avg_pts_against,B_recent_avg_pts_for,B_recent_avg_score_diff,B_recent_losses,B_recent_pts_against_loss,B_recent_pts_against_win,B_recent_pts_for_loss,B_recent_pts_for_win,B_recent_win_ratio,B_recent_wins,B_std_loss_diff,B_std_pts_against_loss,B_std_pts_against_win,B_std_pts_for_loss,B_std_pts_for_win,B_std_win_diff,B_win_ratio,NumOT,Season,score_diff,win
0,0.454545,12,0.200000,3,N,0,X,9,1116,8.083333,67.083333,58.619048,59.000000,68.952381,10.333333,8,0.333333,4,1,0.909091,10,20,35,3,0.700000,7,12,21,0,0.000000,0.0,0,61.500000,74.166667,12.666667,2.0,138.0,231.0,132.0,313.0,0.666667,4.0,7.501010,9.848473,8.570159,12.526336,10.589977,7.558659,0.636364,0.366667,...,0.090909,1,0,X,8,1234,5.500000,62.200000,57.800000,56.700000,76.250000,18.450000,6,0.333333,3,3,0.833333,15,20,49,1,0.666667,2,10,20,0,0.000000,0.0,0,63.833333,62.500000,-1.333333,4.0,251.0,132.0,218.0,157.0,0.333333,2.0,5.562773,8.024961,12.547174,6.147267,14.519950,13.359306,0.666667,0,1985,9,1
1,0.586207,11,0.352941,6,N,0,Z,11,1120,9.636364,76.181818,60.833333,66.545455,72.666667,11.833333,6,0.454545,5,4,0.666667,8,19,42,1,0.833333,5,11,18,1,0.068966,0.5,1,57.000000,63.166667,6.166667,1.0,78.0,264.0,73.0,306.0,0.833333,5.0,7.046598,13.242494,10.325582,11.129813,14.796661,12.015921,0.620690,0.400000,...,0.200000,2,0,Z,6,1345,14.750000,81.875000,57.529412,67.125000,70.058824,12.529412,4,0.600000,6,4,0.714286,10,43,31,0,1.000000,1,8,17,0,0.000000,0.0,0,68.833333,66.333333,-2.500000,2.0,172.0,241.0,111.0,287.0,0.666667,4.0,12.395276,8.288331,9.083761,12.368595,9.555565,8.397303,0.680000,0,1985,1,1
2,0.793103,18,0.217391,5,N,0,W,16,1250,10.833333,71.111111,68.727273,60.277778,74.727273,6.000000,7,0.363636,4,8,0.333333,4,31,18,3,0.500000,3,18,11,0,0.034483,1.0,1,70.000000,69.500000,-0.500000,1.0,75.0,345.0,58.0,359.0,0.833333,5.0,10.205247,9.310897,6.987001,7.168604,9.498325,6.115554,0.379310,0.148148,...,0.500000,2,0,W,1,1207,1.500000,65.500000,59.640000,64.000000,76.680000,17.040000,1,0.857143,6,1,0.923077,12,2,41,0,1.000000,7,2,25,0,0.000000,0.0,0,64.333333,83.666667,19.333333,0.0,0.0,386.0,0.0,502.0,1.000000,6.0,0.707107,0.707107,10.934959,1.414214,11.918893,10.043572,0.925926,0,1985,-25,0
3,0.407407,7,0.363636,4,N,0,Y,9,1229,9.428571,73.571429,62.850000,64.142857,74.200000,11.350000,4,0.600000,6,3,0.785714,11,13,31,0,1.000000,3,7,20,0,0.000000,0.0,0,64.833333,72.000000,7.166667,2.0,148.0,241.0,130.0,302.0,0.666667,4.0,4.353433,8.263517,11.202796,7.174691,11.445983,8.273452,0.740741,0.535714,...,0.400000,6,0,Y,8,1425,8.111111,66.888889,63.526316,58.777778,72.947368,9.421053,3,0.727273,8,5,0.642857,9,21,33,1,0.666667,2,9,19,0,0.000000,0.0,0,68.166667,68.833333,0.666667,3.0,217.0,192.0,193.0,220.0,0.500000,3.0,7.474029,8.950481,8.656202,9.16212

In [11]:
# X and y
X = fcomp.drop(['A_Region', 'B_Region', 'A_TeamID', 'B_TeamID', 'A_away_losses', 'A_away_wins', 'A_home_losses', 'A_home_wins', 'A_neutral_losses', 'A_neutral_wins', 'B_away_losses', 
                'B_away_wins', 'B_home_losses', 'B_home_wins', 'B_neutral_losses', 'B_neutral_wins', 'A_num_losses', 'A_num_wins', 'B_num_losses', 'B_num_wins', 'A_ot_losses', 
                'A_ot_wins', 'B_ot_losses', 'B_ot_wins', 'A_recent_losses', 'A_recent_wins', 'B_recent_losses', 'B_recent_wins', 'NumOT', 'Season', 'score_diff', 'win'], axis=1)
y_regression = fcomp['score_diff']
y_binary = fcomp['win']

In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4034 entries, 0 to 4033
Data columns (total 69 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   A_1_pos_game_ratio         4034 non-null   float64
 1   A_1_pos_losses             4034 non-null   int64  
 2   A_1_pos_win_ratio          4034 non-null   float64
 3   A_1_pos_wins               4034 non-null   int64  
 4   A_Loc                      4034 non-null   object 
 5   A_PlayIn                   4034 non-null   int64  
 6   A_Seed                     4034 non-null   int64  
 7   A_avg_loss_diff            4034 non-null   float64
 8   A_avg_pts_against_loss     4034 non-null   float64
 9   A_avg_pts_against_win      4034 non-null   float64
 10  A_avg_pts_for_loss         4034 non-null   float64
 11  A_avg_pts_for_win          4034 non-null   float64
 12  A_avg_win_diff             4034 non-null   float64
 13  A_away_win_ratio           4034 non-null   float

In [15]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y_regression, test_size=0.2, random_state=SEED)

# scale data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# linear regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

# metrics
print('Linear Regression')
print('MSE:', mean_squared_error(y_test, y_pred))
print('R2:', r2_score(y_test, y_pred))

ValueError: could not convert string to float: 'N'